In [ ]:
import nltk.tag
from nltk.tag.brill import *
import nltk.tag.brill_trainer as bt
from nltk.corpus import brown
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('brown')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

In [ ]:
Template._cleartemplates()
templates = fntbl37()
tagged_sentences = brown.tagged_sents(categories = 'news')
tagged_sentences = tagged_sentences[:]
tagger = nltk.tag.BigramTagger(tagged_sentences)
tagger = bt.BrillTaggerTrainer(tagger, templates, trace=3)
tagger = tagger.train(tagged_sentences, max_rules=250)
print(tagger.evaluate(brown.tagged_sents(categories='fiction')[:]))

TBL train (fast) (seqs: 4623; tokens: 100554; tpls: 37; min score: 2; min acc: None)
Finding initial useful rules...
    Found 365168 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
26372637   015143  | None->NN if Pos:None@[1]
12621262   0  30  | None->. if Pos:NN@[-3,-2,-1]
11931193   0   3  | NN->AT if Word:the@[0] & Pos:NN@[1]
 905 905   0   1  | NN->, if Word:,@[0]
 568 598  30 338  | NN->IN if Pos:AT@[1] & Pos:NN@[2] & Word:the@[1]
 516 574  58   2  | TO->IN if Pos:NN@[1,2]
 427 427   0   5  | NN->CC if Word:and@[0]
 379 379   0   0  | NN->AT if Word:a@[0]
 359 359   0  21  | NN->IN if Word:of@[0]
 243 243   0 118  | NN->TO if Wo

<ipython-input-4-ba7efcfc571b>:8: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  print(tagger.evaluate(brown.tagged_sents(categories='fiction')[:]))


0.7058024763462213


## **Теггинг документа и его обучение на русском языке**

In [ ]:
tagged_sents, tagged_sent = [], []

with open("GSD_train.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.split("\t")
        if len(line) > 1:
            tagged_sent.append((line[1], line[3]))
        else:
            tagged_sents.append(tagged_sent)
            tagged_sent = []
            continue

print(tagged_sents)

[[('Начальный', 'ADJ'), ('ролик', 'NOUN'), (',', 'PUNCT'), ('или', 'CCONJ'), ('опенинг', 'NOUN'), ('(', 'PUNCT'), ('от', 'ADP'), (',', 'PUNCT'), ('сокр.', 'ADV'), (':', 'PUNCT'), ('OP', 'X'), (')', 'PUNCT'), (',', 'PUNCT'), ('как', 'ADP'), ('правило', 'NOUN'), (',', 'PUNCT'), ('представляет', 'VERB'), ('собой', 'PRON'), ('анимацию', 'NOUN'), (',', 'PUNCT'), ('изображающую', 'VERB'), ('главных', 'ADJ'), ('героев', 'NOUN'), ('аниме', 'NOUN'), ('и', 'CCONJ'), ('отражающую', 'VERB'), ('его', 'DET'), ('стиль', 'NOUN'), ('.', 'PUNCT')], [('Стропило', 'NOUN'), (',', 'PUNCT'), ('означающее', 'VERB'), ('победителя', 'NOUN'), (',', 'PUNCT'), ('окрашено', 'VERB'), ('в', 'ADP'), ('красный', 'ADJ'), ('цвет', 'NOUN'), ('--', 'PUNCT'), ('цвет', 'NOUN'), ('гербового', 'ADJ'), ('щита', 'NOUN'), ('Москвы', 'PROPN'), (',', 'PUNCT'), ('стропило', 'NOUN'), (',', 'PUNCT'), ('означающее', 'VERB'), ('побеждённого', 'VERB'), (',', 'PUNCT'), ('имеет', 'VERB'), ('зелёный', 'ADJ'), ('цвет', 'NOUN'), ('--', 'PUNCT

In [ ]:
Template._cleartemplates()
templates = fntbl37()
tagged_sents = tagged_sents[:]
tagger_ru = nltk.tag.BigramTagger(tagged_sents)
tagger_ru = bt.BrillTaggerTrainer(tagger_ru, templates, trace=3)
tagger_ru = tagger_ru.train(tagged_sents, max_rules=250)

TBL train (fast) (seqs: 2038; tokens: 39910; tpls: 37; min score: 2; min acc: None)
Finding initial useful rules...
    Found 73931 useful rules.

           B      |
   S   F   r   O  |        Score = Fixed - Broken
   c   i   o   t  |  R     Fixed = num tags changed incorrect -> correct
   o   x   k   h  |  u     Broken = num tags changed correct -> incorrect
   r   e   e   e  |  l     Other = num tags changed incorrect -> incorrect
   e   d   n   r  |  e
------------------+-------------------------------------------------------
 816 816   02074  | None->NOUN if Pos:None@[1]
 236 236   0   0  | NOUN->PUNCT if Word:,@[0] & Pos:NOUN@[1]
 225 225   0   0  | None->PUNCT if Pos:NOUN@[-1]
  76  76   0  11  | NOUN->CCONJ if Word:и@[0] & Pos:NOUN@[1]
  73  73   0   0  | NOUN->ADP if Word:в@[0] & Pos:NOUN@[1]
  54  54   0   0  | NOUN->PUNCT if Word:)@[0]
  37  37   0   0  | NOUN->PUNCT if Word:&#39;&#39;@[0]
  33  33   0   0  | NOUN->PUNCT if Word:(@[0]
  31  31   0   0  | NOUN->ADP if Word:н

## **Далее тестирование**

In [ ]:
correct, incorrect = 0, 0

with open("GSD_test.txt", "r", encoding="utf-8") as f:
    for line in f:
        line = line.split("\t")
        if len(line) > 1:
            token = line[1]
            gold_tag = line[3]
            model_tag = tagger_ru.tag([token])
            if gold_tag == model_tag[0][1]:
                correct += 1
                #print(gold_tag, model_tag[0][1])
            else:
                incorrect += 1

all_tokens = correct + incorrect
correct_ratio = correct / all_tokens
print("Общее число токенов - ", incorrect)
print("Доля правильно размеченных токенов от общего числа - ", correct_ratio)

Общее число токенов -  966
Доля правильно размеченных токенов от общего числа -  0.07025986525505294
